In [1]:
import pandas as pd 
import numpy as np

import wrds

import statsmodels.api as sm
from scipy.stats import skew, kurtosis, pearsonr, spearmanr
from tabulate import tabulate

c:\Users\YeonChan Kang\anaconda3\envs\fdb\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Ch7. The CRSP Sample and Market Factor

Cause the data is needed for portfolio analysis, we start from the Ch7.   
CRSP data is neccesary for financial studieds 1925 through the present.   
In the book, as in almost all studies of the U.S. stock market, we use the data from CRSP as our primary source for U.S. stock market data.

In [2]:
db = wrds.Connection() # Connect to WRDS

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


## We splite the data sample along two dimension: Stock exchnge and Sector.   

### Decomposing by Stock Exchange

EXCHCD is the exchange code   
1 = NYSE   
2 = AMEX   
3 = NASDAQ   
4 = ARCA   
   
10 = Boston Stock Exchange   
13 = Chicago stock Exxchange   
16 = Pacific Stock Exchange   
17 = Philadelphia Stock Exchange   
19 = Toronto Stock Exchange   
20 = OTC   
   
-2 = delisted   
-1 = suspended   
0 = not NYSE, AMEX, or NASDAQ   

In [3]:
db.describe_table(library='crsp', table='msf')

Approximately 5037353 rows in crsp.msf.


,name,nullable,type,comment
0,cusip,True,VARCHAR(8),None
1,permno,True,INTEGER,None
2,permco,True,INTEGER,None
3,issuno,True,INTEGER,None
4,hexcd,True,SMALLINT,None
5,hsiccd,True,INTEGER,None
6,date,True,DATE,None
7,bidlo,True,"NUMERIC(11, 5)",None
8,askhi,True,"NUMERIC(11, 5)",None
9,prc,True,"NUMERIC(11, 5)",None


Before 1963 July, CRSP doesn't contain AMEX.   
1972 Nov, Nasdaq added at CRSP but NASDAQ stock appears in January 1969.   

### Decomposting by industry sector

These securities can be identified using CRSP’s monthly stock names (msenames) file.   
SHRCD = 10 or 11 is common stock   
SICCD code standard industrial classification code   
SIC CODE 0001-0999: Agriculture, Forestry and Fishing   
SIC CODE 1000-1499: Mining   
SIC CODE 1500-1799: Construction   
SIC CODE 2000-3999: Manufacturing   
SIC CODE 4000-4999: Transportation, Communications, Electric, Gas and Sanitary service   
SIC CODE 5000-5199: Wholesale Trade   
SIC CODE 5200-5999: Retail Trade   
SIC CODE 6000-6799: Finance, Insurance and Real Estate   
SIC CODE 7000-8999: Services   
SIC CODE 9100-9729: Public Administration   

In [4]:
db.describe_table(library='crsp', table='msenames') 

Approximately 113856 rows in crsp.msenames.


,name,nullable,type,comment
0,permno,True,INTEGER,None
1,namedt,True,DATE,None
2,nameendt,True,DATE,None
3,shrcd,True,SMALLINT,None
4,exchcd,True,SMALLINT,None
5,siccd,True,INTEGER,None
6,ncusip,True,VARCHAR(8),None
7,ticker,True,VARCHAR(8),None
8,comnam,True,VARCHAR(35),None
9,shrcls,True,VARCHAR(4),None


## Stock returns and Excess return

In many cases, it is optimal to examine excess stock returns instead of simply the stock return.   
Follow the book, we use risk-free rate from Kenneth French data   
We express excess return as $r_{i,t}$   
   
For the most part, however, the cross-sectional distributions of unadjusted returns look very similar to the distributions of delisting-adjusted returns.   
It is not surprising, therefore, that the choice of whether to use the unadjusted or adjusted returns has very little impact on the results of most empirical analyses.   
That being said, to more accurately reflect the returns an investor would realize by making an investment at the end of month t and holding until the end of month t + 1, we use the delisting-adjusted future excess return (rt+1) as the focal return variable throughout this text.

In [5]:
# Kenneth French Data Library
db.describe_table(library='ff', table='factors_monthly')

Approximately 1174 rows in ff.factors_monthly.


,name,nullable,type,comment
0,date,True,DATE,None
1,mktrf,True,"NUMERIC(8, 6)",None
2,smb,True,"NUMERIC(8, 6)",None
3,hml,True,"NUMERIC(8, 6)",None
4,rf,True,"NUMERIC(7, 5)",None
5,year,True,DOUBLE PRECISION,None
6,month,True,DOUBLE PRECISION,None
7,umd,True,"NUMERIC(8, 6)",None
8,dateff,True,DATE,None


In [6]:
# Kenneth French Data Library
db.describe_table(library='ff', table='factors_daily')

Approximately 25732 rows in ff.factors_daily.


,name,nullable,type,comment
0,date,True,DATE,None
1,mktrf,True,"NUMERIC(8, 6)",None
2,smb,True,"NUMERIC(8, 6)",None
3,hml,True,"NUMERIC(8, 6)",None
4,rf,True,"NUMERIC(7, 5)",None
5,umd,True,"NUMERIC(8, 6)",None


In [7]:
# Get only risk-free rate, market excess return
rf_m = db.raw_sql("""SELECT 
                        date, rf, mktrf
                    FROM 
                        ff.factors_monthly
                    WHERE date between '1988-01-01' and '2012-12-31' 
                """)

In [8]:
rf_m

,date,rf,mktrf
0,1988-01-01,0.0029,0.0421
1,1988-02-01,0.0046,0.0475
2,1988-03-01,0.0044,-0.0227
3,1988-04-01,0.0046,0.0056
4,1988-05-01,0.0051,-0.0029
...,...,...,...
295,2012-08-01,0.0001,0.0255
296,2012-09-01,0.0001,0.0273
297,2012-10-01,0.0001,-0.0176
298,2012-11-01,0.0001,0.0078


In [9]:
rf_d = db.raw_sql("""SELECT date, rf, mktrf
                    FROM ff.factors_daily
                    WHERE date between '1988-01-01' and '2012-12-31' 
                """)

In [10]:
rf_d

,date,rf,mktrf
0,1988-01-04,0.00015,0.0334
1,1988-01-05,0.00015,0.0122
2,1988-01-06,0.00015,0.0027
3,1988-01-07,0.00015,0.0074
4,1988-01-08,0.00015,-0.0566
...,...,...,...
6297,2012-12-24,0.00001,-0.0024
6298,2012-12-26,0.00001,-0.0054
6299,2012-12-27,0.00001,-0.0011
6300,2012-12-28,0.00001,-0.0100


## Delisted Stock

In many cases, it is optimal to examine excess stock returns instead of simply the stock return.
We can use risk-free rate from Kenneth French data
We express excess return as $r_{i,t}$

For the most part, however, the cross-sectional distributions of unadjusted returns look very similar to the distributions of delisting-adjusted returns. It is not surprising, therefore, that the choice of whether to use the unadjusted or adjusted returns has very little impact on the results of most empirical analyses. That being said, to more accurately reflect the returns an investor would realize by making an investment at the end of month t and holding until the end of month t + 1, we use the delisting-adjusted future excess return (rt+1) as the focal return variable throughout this text.

## Identifier and characteristic

Get the description of the table crsp.msf   
This table contains monthly stock data for all CRSP-listed securities.   
   
The most used columns are:   
- SHROUT is the number of shares outstanding in thousands
- ALTPRC is the adjusted price. Take absolute value of ALTPRC to get the price 
- MarketCap = SHROUT * ALTPRC / 1000
- Ret is the return of the stock
The exceptions to this are the monthly return when a stock delists from an exchange.   
   
I usally we only consider stocks with EXCHCD = 1, 2, 3, and the others are considered as 0
db.describe_table(library='crsp', table='msf')   

In [11]:
db.describe_table(library='crsp', table='msedelist')

Approximately 37747 rows in crsp.msedelist.


,name,nullable,type,comment
0,permno,True,INTEGER,None
1,dlstdt,True,DATE,None
2,dlstcd,True,SMALLINT,None
3,nwperm,True,INTEGER,None
4,nwcomp,True,INTEGER,None
5,nextdt,True,DATE,None
6,dlamt,True,"NUMERIC(11, 5)",None
7,dlretx,True,"NUMERIC(10, 6)",None
8,dlprc,True,"NUMERIC(11, 5)",None
9,dlpdt,True,DATE,None


### More detail

#### Identifier
- Cusip  (VARCHAR(8))       : 8-character identifier for securities, with issuer (6 chars) and issue (2 chars), including dummy codes assigned by CRSP.   
- PERMCO (INTEGER)          : Unique, permanent identifier for companies on CRSP files, with values under 20,000 indicating Nasdaq-assigned numbers.   
- ISSUNO (INTEGER)          : Unique identifier for each Nasdaq-listed security, set to zero if unknown, may change if Nasdaq reassigns the number.   
- HEXCD  (INTEGER)          : Latest exchange code for a security, with 1 for NYSE, 2 for AMEX, and 3 for Nasdaq.   
- HSICCD (INTEGER)          : Latest non-zero Standard Industrial Classification (SIC) Code for a security; zero if not provided by CRSP's data source.   

##### Date
- DATE (DATE)               : Date of the observation, in the form YYYY-MM-DD.

#### Characteristics
- PRC   (NUMERIC(11,5))     : Closing price or negative bid/ask average for a trading day; set to zero if unavailable; monthly data shows last trading date price.   
- VOL   (NUMERIC(10,0))     : Trading volume, expressed in units of one share for daily data and hundreds for monthly data, set to -99 if missing, with zero indicating no trades.   
- RET   (NUMERIC(10,6))     : Holding period return, calculated as (p(t)f(t)+d(t))/p(t')-1, with special codes indicating reasons for missing returns.   
- BID   (NUMERIC(11,5))     : Closing bid price for NYSE, AMEX, and NASDAQ securities; NASDAQ uses inside quotations since July 1980, while NYSE/AMEX uses the last representative quote.   
- ASK   (NUMERIC(11,5))     : Closing ask price for NYSE, AMEX, and NASDAQ securities; NASDAQ uses inside quotations since July 1980, while NYSE/AMEX uses the last representative quote.   
- SHROUT (DOUBLE PRECISION) : Number of publicly held shares, recorded in thousands.   
- BIDLO (NUMERIC(11,5))     : Daily lowest trading or closing bid price, set to zero if unavailable; monthly files show lowest daily price or bid/ask average.   
- ASKHI (NUMERIC(11,5))     : Daily highest trading or closing ask price, set to zero if unavailable; monthly files show highest daily price or bid/ask average.   
- CFACPR(DOUBLE PRECISION)  : Cumulative factor to adjust price.   
- CFACSHR(DOUBLE PRECISION) : Cumulative factor to adjust shares outstanding.   
- ALTPRC(NUMERIC(11,5))     : Alternate monthly price from daily prices, showing the last non-missing price in the month; available only on monthly databases.   
- SPREAD(NUMERIC(10,5))     : Monthly difference between closing bid and ask quotes, set to zero if unavailable, with sign indicating bid/ask status when Closing Price or Bid/Ask Average is zero.   
- ALTPRCDT(DATE)            : Date of the monthly Price Alternate; set to zero if no non-missing prices in the month.   
- RETX(NUMERIC(10,6))       : Holding period return excluding dividends, calculated similarly to RET but typically with dividends (d(t)) set to zero.   

In [12]:
# Use altprc instead of prc to get the adjusted price
sql = """SELECT date, permno, altprc, ret, shrout, vol
        FROM crsp.msf 
        WHERE date between '1988-01-01' and '2012-12-31'
        """
crsp_m = db.raw_sql(sql, date_cols=['date'], chunksize=100000)

In [13]:
crsp_m

,date,permno,altprc,ret,shrout,vol
0,1988-01-29,10001,6.2500,0.063830,992.0,490.0
1,1988-02-29,10001,6.7500,0.080000,992.0,382.0
2,1988-03-31,10001,6.1250,-0.076296,992.0,369.0
3,1988-04-29,10001,-6.3125,0.030612,992.0,121.0
4,1988-05-31,10001,-6.4375,0.019802,992.0,102.0
...,...,...,...,...,...,...
74724,2012-08-31,93436,28.5200,0.040117,105432.0,247781.0
74725,2012-09-28,93436,29.2800,0.026648,105772.0,335868.0
74726,2012-10-31,93436,28.1314,-0.039228,113779.0,178360.0
74727,2012-11-30,93436,33.8200,0.202215,113779.0,224326.0


In [ ]:
# There are no altprc in crsp.dsf
crsp_d = db.raw_sql("""SELECT date, permno, prc, ret, shrout, vol
                      FROM crsp.dsf 
                      WHERE date between '1988-01-01' and '2012-12-31'
                  """) 

In [ ]:
crsp_d

### Preprocessing

In [24]:
def convert_types(df):
    """
    Optimizes the data types of a DataFrame.
    
    Args:
        df (pd.DataFrame): The DataFrame to be optimized.
    
    Returns:
        pd.DataFrame: The DataFrame with optimized data types.
    """
    changes = []

    for col in df.columns:
        original_dtype = df[col].dtype
        
        # Convert date columns to datetime type
        if pd.api.types.is_datetime64_any_dtype(df[col]) or 'date' in col.lower():
            df[col] = pd.to_datetime(df[col], errors='coerce')
        
        # Convert integer-like columns to appropriate integer type
        elif pd.api.types.is_integer_dtype(df[col]):
            df[col] = pd.to_numeric(df[col], downcast='integer', errors='coerce')
            # Downcast to smallest possible integer type
            if df[col].max(skipna=True) <= np.iinfo(np.int16).max and df[col].min(skipna=True) >= np.iinfo(np.int16).min:
                df[col] = df[col].astype(pd.Int16Dtype())
            elif df[col].max(skipna=True) <= np.iinfo(np.int32).max and df[col].min(skipna=True) >= np.iinfo(np.int32).min:
                df[col] = df[col].astype(pd.Int32Dtype())
        
        # Convert float-like columns to appropriate float type
        elif pd.api.types.is_float_dtype(df[col]):
            df[col] = pd.to_numeric(df[col], downcast='float', errors='coerce')
            
            # Check if all values are integers (float ending with .0)
            finite_values = df[col][np.isfinite(df[col])]
            if all(finite_values == finite_values.astype(np.int64)):
                df[col] = df[col].astype(pd.Int64Dtype())
                # Downcast to smallest possible integer type
                if df[col].max(skipna=True) <= np.iinfo(np.int16).max and df[col].min(skipna=True) >= np.iinfo(np.int16).min:
                    df[col] = df[col].astype(pd.Int16Dtype())
                elif df[col].max(skipna=True) <= np.iinfo(np.int32).max and df[col].min(skipna=True) >= np.iinfo(np.int32).min:
                    df[col] = df[col].astype(pd.Int32Dtype())
            else:
                # Downcast to smallest possible float type
                if df[col].max(skipna=True) <= np.finfo(np.float16).max and df[col].min(skipna=True) >= np.finfo(np.float16).min:
                    df[col] = df[col].astype(np.float16)
                elif df[col].max(skipna=True) <= np.finfo(np.float32).max and df[col].min(skipna=True) >= np.finfo(np.float32).min:
                    df[col] = df[col].astype(np.float32)
        
        # Convert object columns to string type
        elif pd.api.types.is_object_dtype(df[col]):
            df[col] = df[col].astype(str)
        
        # Record changes in column data types
        if df[col].dtype != original_dtype:
            changes.append(f"'{col}' changed from {original_dtype} to {df[col].dtype}")
    
    # Print changes in column data types
    if changes:
        for change in changes:
            print(change)
    else:
        print("No changes were made.")
    
    return df

In [15]:
crsp_m['altprc'] = crsp_m['altprc'].abs() # abs(prc) If there is not a close price, the mid price of bid and ask is used with a negative sign. Change them to a positive value.
crsp_m['mcap'] = crsp_m.altprc * crsp_m.shrout # Market capitalization
crsp_m['size'] = np.log(crsp_m['mcap']) # Size is the log of market capitalization

c:\Users\YeonChan Kang\anaconda3\envs\fdb\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [16]:
crsp_m

,date,permno,altprc,ret,shrout,vol,mcap,size
0,1988-01-29,10001,6.2500,0.063830,992.0,490.0,6.200000e+03,8.732305
1,1988-02-29,10001,6.7500,0.080000,992.0,382.0,6.696000e+03,8.809266
2,1988-03-31,10001,6.1250,-0.076296,992.0,369.0,6.076000e+03,8.712102
3,1988-04-29,10001,6.3125,0.030612,992.0,121.0,6.262000e+03,8.742255
4,1988-05-31,10001,6.4375,0.019802,992.0,102.0,6.386000e+03,8.761863
...,...,...,...,...,...,...,...,...
74724,2012-08-31,93436,28.5200,0.040117,105432.0,247781.0,3.006921e+06,14.916427
74725,2012-09-28,93436,29.2800,0.026648,105772.0,335868.0,3.097004e+06,14.945946
74726,2012-10-31,93436,28.1314,-0.039228,113779.0,178360.0,3.200763e+06,14.978900
74727,2012-11-30,93436,33.8200,0.202215,113779.0,224326.0,3.848006e+06,15.163066


In [25]:
crsp_m = convert_types(crsp_m)

No changes were made.


In [43]:
# There are so many missing values in the dataset.
# We use the dropna method to remove the missing values.
crsp_m.dropna(inplace=True)

In [44]:
crsp_m

,date,permno,altprc,ret,shrout,vol,mcap,size
0,1988-01-29,10001,6.250000,0.063843,992,490,6200.00,8.732305
1,1988-02-29,10001,6.750000,0.080017,992,382,6696.00,8.809265
2,1988-03-31,10001,6.125000,-0.076294,992,369,6076.00,8.712102
3,1988-04-29,10001,6.312500,0.030609,992,121,6262.00,8.742255
4,1988-05-31,10001,6.437500,0.019806,992,102,6386.00,8.761864
...,...,...,...,...,...,...,...,...
74724,2012-08-31,93436,28.520000,0.040131,105432,247781,3006920.75,14.916427
74725,2012-09-28,93436,29.280001,0.026642,105772,335868,3097004.25,14.945946
74726,2012-10-31,93436,28.131399,-0.039215,113779,178360,3200762.50,14.978900
74727,2012-11-30,93436,33.820000,0.202271,113779,224326,3848005.75,15.163066


## Ch1. Preliminaries

### Winsorization and Truncation

Empirical asset pricing researchers usually take a more ad hoc approach to dealing with the effect of outliers   

Two techniques are commonly used in empirical asset pricing research to deal with the effect of outliers: winsorizing and truncating.    

#### Winsorization

Winsorizing is a technique that replaces extreme values with the nearest non-extreme value.   
The idea is to replace the extreme values with the nearest non-extreme value.   
For example, if the 1% quantile is 0.5 and the 99% quantile is 100, then all values less than 0.5 are replaced with 0.5 and all values greater than 100 are replaced with 100.   


In [29]:
def winsorize(data, column=False, lower=0.01, upper=0.99, copy=True):
    """
    Winsorizes the input data by replacing extreme values with the nearest values within the specified quantiles.
    
    Parameters:
    data (pd.Series or pd.DataFrame): The data to be winsorized.
    lower (float): The lower quantile threshold. Defaults to 0.01.
    upper (float): The upper quantile threshold. Defaults to 0.99.
    copy (bool): Whether to return a copy of the data or to modify it in place. Defaults to True.
    
    Returns:
        pd.Series or pd.DataFrame: The winsorized data.
    """
    print(f"Data winsorized between {lower} and {upper}\n")
    
    if copy:
        data = data.copy()
    
    if column:
        data = data[column]
    
    print(f"Original maximum: {data.max()} and Original minimum: {data.min()}\n")
    
    qtl = data.quantile([lower, upper])
    
    # Replace values below3 the lower quantile
    data[data < qtl.loc[lower]] = qtl.loc[lower]
    
    # Replace values above the upper quantile
    data[data > qtl.loc[upper]] = qtl.loc[upper]
    
    
    print(f"New maximum: {data.max()} and New minimum: {data.min()}\n")
    return data

In [30]:
winsor_data = crsp_m.copy()
winsor_data['ret'] = winsorize(crsp_m['ret']) # Winsorize returns

Data winsorized between 0.01 and 0.99

Original maximum: 24.0 and Original minimum: -0.98828125

New maximum: 0.5966796875 and New minimum: -0.41015625



#### Truncating

Truncating is a technique that removes extreme values.   
The idea is to remove the extreme values from the data.   
For example, if the 1% quantile is 0.5 and the 99% quantile is 100, then all values less than 0.5 and all values greater than 100 are removed.   

In [35]:
def truncate(data, column=False, lower=0.01, upper=0.99, copy=True):
    """

    Args:
        data (_type_): The data to be truncated.
        lower (float): The lower quantile threshold. Defaults to 0.01.
        upper (float): The upper quantile threshold. Defaults to 0.99.
        copy (bool): Whether to return a copy of the data or to modify it in place. Defaults to True.
    
    Returns:
        pd.Series or pd.DataFrame: The truncated data.
    """
    print(f"Data winsorized between {lower} and {upper}\n")
    
    if copy:
        data = data.copy()
        
    if column:
        data = data[column]
    
    print(f"Original maximum: {data.max()} and Original minimum: {data.min()}\n")
    
    qtl = data.quantile([lower, upper])
    
    # Remove values below the lower quantile
    data = data[data >= qtl.loc[lower]]
    
    # Remove values above the upper quantile
    data = data[data <= qtl.loc[upper]]
    
    print(f"New maximum: {data.max()} and New minimum: {data.min()}\n")
    return data

In [36]:
truncate_data = crsp_m.copy()
truncate_data['ret'] = truncate(crsp_m, ['ret']) # Winsorize returns

Data winsorized between 0.01 and 0.99

Original maximum: ret    24.0
dtype: float16 and Original minimum: ret   -0.988281
dtype: float16

New maximum: ret    0.59668
dtype: float16 and New minimum: ret   -0.410156
dtype: float16



### NEWEY AND WEST (1987) ADJUSTMENT

Newey-West standard errors are a robust method for estimating the standard errors of the coefficients in a regression model.   
The selection of the lag length is important because it determines the number of periods over which the autocorrelation of the residuals is calculated.   
Usally, the lag length is set to the integer part of the cube root of the number of observations or 6 or 12 in empirical asset pricing research.   
The Newey-West standard errors are calculated as follows:
- 1. Estimate the regression model.
- 2. Calculate the residuals.
- 3. Calculate the autocovariance of the residuals.
- 4. Calculate the Newey-West standard errors.
   
The Newey-West standard errors are robust to autocorrelation and heteroskedasticity in the residuals.


In [61]:
def newey_west_regression(y, X, lags=None):
    """
    Performs a regression and calculates Newey-West standard errors, t-statistics, and p-values.
    
    Args:
        y (pd.Series or np.array): Dependent variable.
        X (pd.DataFrame or np.array): Independent variables.
        lags (int): The lag length for Newey-West standard errors. If None, it will be set to the integer part of the cube root of the number of observations.
    
    Returns:
        pd.DataFrame: A DataFrame containing coefficients, Newey-West standard errors, t-statistics, and p-values.
    """
    if lags is None:
        lags = int(np.floor(len(y)**(1/3)))  # Set lag length to the integer part of the cube root of the number of observations

    # Ensure the dependent variable is numeric
    y = pd.to_numeric(y, errors='coerce')
    
    # Ensure all independent variables are numeric
    X = X.apply(pd.to_numeric, errors='coerce')

    # Drop rows with any NaN values in y or X
    valid_idx = ~y.isna() & X.notna().all(axis=1)
    y = y[valid_idx]
    X = X[valid_idx]
    
    # Add a constant term to the independent variables matrix
    X = sm.add_constant(X)
    
    # Estimate the regression model
    model = sm.OLS(y, X).fit()
    
    # Calculate Newey-West standard errors
    robust_cov = model.get_robustcov_results(cov_type='HAC', maxlags=lags)
    
    # Extract coefficients, standard errors, t-statistics, and p-values
    results = pd.DataFrame({
        'Coefficient': model.params,
        'Newey-West SE': robust_cov.bse,
        't-Statistic': robust_cov.tvalues,
        'p-Value': robust_cov.pvalues
    })
    
    return results

In [66]:
a = crsp_m.drop(['ret', 'date', 'permno'], axis=1)

In [67]:
newey_west_regression(crsp_m['ret'], a, lags=6)

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

## Ch2. Summary Statistics

In [8]:
# The summary statistics procedure consists of two sterps: 
# 1. For each time period $t$, certain charaacteristics of cross-sectional distribution of the given variable, $X$, are calculated.
# 2. The time series properties of the periodic cross-sectional characteristics are calculated. 
#   The most importanr time series properties are the mean
#   The mean means the average value of the cross-sectional characteristic over time.

# Statistics for univariate distributions of the varaibles used in a study

In [ ]:
#The details of the first step are as follows. For each time period t, we calculate the cross-sectional mean, standard deviation, skewness, excess kurtosis, minimum value, median value, maximum value, and selected additional 
#percentiles of the distribution of the values of X, where each of these statistics is calculated over all available values of X in period t. 
#We let Meant be the mean, SDt denote the sample standard deviation, Ske𝑤t represent the sample skewness, Kurtt be the sample excess kurtosis, Mint be the minimum value, Mediant denote the median value, and Maxt represent the 
#maximum value of X in period t. In addition, we will record the fifth, 25th, 75th, and 95th percentiles of X in month t, which we denote P5t, P25t, P75t, and P95t, respectively. 
#Depending on the data and the objective of the study, it may be desirable to include additional percentiles of the distribution. 
#For example, if the study focuses on extreme values of X, then it may be valuable to record the first, second, third, fourth, 96th, 97th, 98th, and 99th percentiles of the distribution as well.
#Alternatively, calculating the minimum, maximum, fifth percentile, and 95th percentile of the data may not be necessary if the data are reasonably well behaved. 
#Exactly which statistics to record and present is a decision made by the researcher, who, presumably, has a much deeper understanding of the data than could possibly be presented in a research article.
#In addition to these statistics describing the time t cross-sectional distribution of X, we also record the number of entities for which a valid value of X is available in period t and denote this number nt.

In [ ]:
# The objectives in analyzing the summary statistics are twofold. First, the summary statistics are intended to give a basic overview of the cross-sectional properties 
# of the variables that will be used in the study. This is useful for understanding the types of entities that comprise the sample. 
# Second, the summary statistics can be used to identify any potential issues that may arise when using these variables in statistical analyses.

First Stage

In [64]:
def cal_cs_stats(df, time_column, value_column, additional_percentiles=False):
    """
    Calculate cross-sectional statistics for each time period.
    
    Args:
        df (pd.DataFrame): The data frame containing the data.
        time_column (str): The name of the column representing time periods.
        value_column (str): The name of the column representing the values of X.
        additional_percentiles (list of float): Additional percentiles to calculate (optional).
    
    Returns:
        pd.DataFrame: A data frame containing the calculated statistics for each time period.
    """
    if additional_percentiles:
        additional_percentiles_list = [0.01, 0.02, 0.03, 0.04, 0.96, 0.97, 0.98, 0.99]

    # Function to calculate the required statistics for a given period
    def calc_period_stats(group, additional_percentiles=additional_percentiles):
        stats = {
            'Time': group[time_column].iloc[0],  # Time column
            'Mean': group[value_column].mean(),
            'SD': group[value_column].std(),
            'Skew': skew(group[value_column], nan_policy='omit'),
            'Kurt': kurtosis(group[value_column], nan_policy='omit'),
            'Min': group[value_column].min(),
            'Median': group[value_column].median(),
            'Max': group[value_column].max(),
            'P5': group[value_column].quantile(0.05),
            'P25': group[value_column].quantile(0.25),
            'P75': group[value_column].quantile(0.75),
            'P95': group[value_column].quantile(0.95),
            'N': group[value_column].count()
        }
        
        if additional_percentiles:
            for percentile in additional_percentiles_list:
                stats[f'P{int(percentile*100)}'] = group[value_column].quantile(percentile)
        
        return pd.Series(stats)
    
    # Group the data by the time column and apply the function
    stats_df = df.groupby(time_column).apply(calc_period_stats).reset_index(drop=True)
    
    
    return stats_df

# Example usage
# Assuming `df` is your DataFrame with 'Date' as the time column and 'Value' as the value column
df = pd.DataFrame({
    'Date': ['2024-01', '2024-01', '2024-01', '2024-02', '2024-02', '2024-02', '2024-02'],
    'Value': [10, 15, 14, 20, 22, 21, 19]
})

results = cal_cs_stats(df, 'Date', 'Value')
print(results)

      Time  Mean        SD     Skew  Kurt  Min  Median  Max     P5    P25  \
0  2024-01  13.0  2.645751 -0.59517 -1.50   10    14.0   15  10.40  12.00   
1  2024-02  20.5  1.290994  0.00000 -1.36   19    20.5   22  19.15  19.75   

     P75    P95  N  
0  14.50  14.90  3  
1  21.25  21.85  4  


C:\Users\YeonChan Kang\AppData\Local\Temp\ipykernel_72644\734699324.py:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stats_df = df.groupby(time_column).apply(calc_period_stats).reset_index(drop=True)


Second Stage

caculate the time-series averages of the periodic cross-sectional values.
    $Mean$: time-series average of the values of $Mean_t$ over all periods $t$ in the sample.

Caculate the time-series means of the corss-sectional summary statistics

In [80]:
def calculate_time_series_averages(stats_df):
    """
    Calculate the time-series averages of the cross-sectional statistics.
    
    Args:
        stats_df (pd.DataFrame): A data frame containing cross-sectional statistics for each time period.
    
    Returns:
        pd.Series: A series containing the time-series averages of the cross-sectional statistics.
    """
    # Exclude the time column for averaging
    stats_to_average = stats_df.drop(columns=['Time'])
    
    # Calculate the time-series averages
    time_series_averages = stats_to_average.mean()
    
    df = pd.DataFrame(time_series_averages).T

    return df

cross_sectional_stats = cal_cs_stats(df, 'Date', 'Value')
time_series_averages = calculate_time_series_averages(cross_sectional_stats)
print("\nTime-Series Averages of Cross-Sectional Statistics:")
print(time_series_averages)


Time-Series Averages of Cross-Sectional Statistics:
    Mean        SD      Skew  Kurt   Min  Median   Max      P5     P25  \
0  16.75  1.968373 -0.297585 -1.43  14.5   17.25  18.5  14.775  15.875   

      P75     P95    N  
0  17.875  18.375  3.5  


C:\Users\YeonChan Kang\AppData\Local\Temp\ipykernel_72644\734699324.py:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stats_df = df.groupby(time_column).apply(calc_period_stats).reset_index(drop=True)


In [83]:
time_series_averages

,Mean,SD,Skew,Kurt,Min,Median,Max,P5,P25,P75,P95,N
0,16.75,1.968373,-0.297585,-1.43,14.5,17.25,18.5,14.775,15.875,17.875,18.375,3.5


In [86]:
print(tabulate(time_series_averages, headers='keys', tablefmt='facy_grid'))

      Mean       SD       Skew    Kurt    Min    Median    Max      P5     P25     P75     P95    N
--  ------  -------  ---------  ------  -----  --------  -----  ------  ------  ------  ------  ---
 0   16.75  1.96837  -0.297585   -1.43   14.5     17.25   18.5  14.775  15.875  17.875  18.375  3.5


## Ch3. Correlation

There are two correalations 
1. Pearson product moment correlation
2. Spearmann rank correlation 

Pearson product moment correlation: most applicable when the relation between the two variabels, which we denote X and Y.
Pearson correlation can be roughly interpreted as the signed percentage of variation in X that is related to variation in Y.
With the sign being positive if X tends to be high when Y is high and the sign being negative whne high values of X tends to correspeond to low values of Y

Spearman rank correlation is most application when the relation between the variables is though to be monotonic but not neccessarily linear.
Mesures how closely related the ordering of X is to the ordering of Y

1. Calcuate the cross-sectional correlation between the two variables in question, X and Y for each period t
2. time-series average of the cross-sectional correlations.

In [181]:
def cal_corr(group, var1, var2, option='all'):
    """
    Calculate the correlation between two variables for a given period.

    Args:
        group (pd.DataFrame): The data for the given period.
        var1 (str): The name of the first variable.
        var2 (str): The name of the second variable.
        option (str): The correlation type to calculate ('pearson', 'spearman', or 'all'). Defaults to 'all'.
        
    Returns:
        pd.Series: A series containing the correlation between the two variables.
    """
    if option == 'pearson':
        pearson_corr, _ = pearsonr(group[var1], group[var2])
        return pd.Series({'Pearson': pearson_corr})
    elif option == 'spearman':
        spearman_corr, _ = spearmanr(group[var1], group[var2])
        return pd.Series({'Spearman': spearman_corr})
    elif option == 'all':
        pearson_corr, _ = pearsonr(group[var1], group[var2])
        spearman_corr, _ = spearmanr(group[var1], group[var2])
        return pd.Series({'Pearson': pearson_corr, 'Spearman': spearman_corr})
    else:
        raise ValueError("Invalid option for correlation type. Choose from 'pearson', 'spearman', or 'all'.")

In [131]:
def cal_per_corr(df, time_column, specific_date=None):
    """
    Calculate Pearson and Spearman correlations for each time period for all pairs of variables.
    
    Args:
        df (pd.DataFrame): The data frame containing the data.
        time_column (str): The name of the column representing time periods.
        specific_date (str or None): A specific date to filter the data. If None, calculate for all dates.
    
    Returns:
        pd.DataFrame: A data frame containing the Pearson and Spearman correlations for each time period.
    """
    if specific_date:
        df = df[df[time_column] == specific_date]
    
    variables = [col for col in df.columns if col != time_column]
    correlations = []

    for i, var1 in enumerate(variables):
        for var2 in variables[i+1:]:
            corr_df = df.groupby(time_column).apply(lambda group: cal_corr(group, var1, var2)).reset_index()
            corr_df['Var1'] = var1
            corr_df['Var2'] = var2
            correlations.append(corr_df)

    all_correlations = pd.concat(correlations, ignore_index=True)
    return all_correlations

In [112]:
def cal_ts_avcorr(correlations_df):
    """
    Calculate the time-series averages of the periodic cross-sectional correlations.
    
    Args:
        correlations_df (pd.DataFrame): A data frame containing the periodic correlations.
    
    Returns:
        pd.DataFrame: A data frame containing the time-series average Pearson and Spearman correlations.
    """
    avg_corrs = correlations_df.groupby(['Var1', 'Var2'])[['Pearson', 'Spearman']].mean().reset_index()
    return avg_corrs

In [113]:
def create_corr_mat(avg_corrs, variables):
    """
    Create a correlation matrix with Pearson correlations below the diagonal and Spearman correlations above the diagonal.
    
    Args:
        avg_corrs (pd.DataFrame): A data frame containing the average correlations.
        variables (list of str): List of column names representing the variables.
    
    Returns:
        pd.DataFrame: A correlation matrix.
    """
    matrix = pd.DataFrame(index=variables, columns=variables)

    for _, row in avg_corrs.iterrows():
        var1, var2 = row['Var1'], row['Var2']
        pearson, spearman = row['Pearson'], row['Spearman']
        matrix.at[var1, var2] = f"{spearman:.2f}"
        matrix.at[var2, var1] = f"{pearson:.2f}"
    
    np.fill_diagonal(matrix.values, np.nan)
    return matrix

In [132]:
df = pd.DataFrame({
    'Date': ['2024-01', '2024-01', '2024-01', '2024-02', '2024-02', '2024-02', '2024-02'],
    'X': [10, 15, 14, 20, 22, 21, 19],
    'Y': [8, 10, 12, 18, 16, 15, 14],
    'Z': [7, 11, 13, 17, 15, 14, 13]
})

variables = ['X', 'Y', 'Z']

In [133]:
# Step 1: Calculate periodic correlations
periodic_correlations = cal_per_corr(df, 'Date')
print("Periodic Correlations:")
print(periodic_correlations)

Periodic Correlations:
      Date   Pearson  Spearman Var1 Var2
0  2024-01  0.755929       0.5    X    Y
1  2024-02  0.226779       0.4    X    Y
2  2024-01  0.866025       0.5    X    Z
3  2024-02  0.226779       0.4    X    Z
4  2024-01  0.981981       1.0    Y    Z
5  2024-02  1.000000       1.0    Y    Z


C:\Users\YeonChan Kang\AppData\Local\Temp\ipykernel_72644\3769392557.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  corr_df = df.groupby(time_column).apply(lambda group: cal_corr(group, var1, var2)).reset_index()
C:\Users\YeonChan Kang\AppData\Local\Temp\ipykernel_72644\3769392557.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  corr_df = df.groupby(time_column).apply(lambda group: cal_corr(group,

In [134]:
# Step 2: Calculate time-series average correlations
average_correlations = cal_ts_avcorr(periodic_correlations)
print("\nTime-Series Average Correlations:")
print(average_correlations)


Time-Series Average Correlations:
  Var1 Var2   Pearson  Spearman
0    X    Y  0.491354      0.45
1    X    Z  0.546402      0.45
2    Y    Z  0.990990      1.00


In [135]:
# Step 3: Create the correlation matrix
correlation_matrix = create_corr_mat(average_correlations, variables)
print("\nCorrelation Matrix:")
print(correlation_matrix)


Correlation Matrix:
      X     Y     Z
X   NaN  0.45  0.45
Y  0.49   NaN  1.00
Z  0.55  0.99   NaN


If the Spearman rank correlation is substantially larger in magnitude than the Pearson product–moment correlation, this likely indicates that there is a monotonic, but not linear, relation between the variables. 
This type of relation signals that linear regression analysis is a potentially problematic statistical technique to apply to the given variables if one of the variables is used as the dependent variable. 

If the Pearson product–moment correlation is substantially larger in magnitude than the Spearman rank correlation, this may indicate that there are a few extreme data points in one of the variables that are exerting a strong influence on the calculation of the Pearson product–moment correlation.
In this case, it is possible that winsorizing one or both of the variables at a higher level will alleviate this issue. 

Finally, it is worth noting here that, because of the assumption of linearity in the calculation of the Pearson product–moment correlation, this measure is usually more indicative of results that will be realized using regression techniques such as Fama and MacBeth (1973) regression analysis (presented in Chapter 6). Because the Spearman rank correlation is based on the ordering of the variables, Spearman rank correlations are more likely indicative of the results of analyses that rely on the ranking, or ordering, of the variables, such as portfolio analysis (presented in Chapter 5).

## Ch4. Persistence Analysis

Many of the variables in empirical asset pricing research are intended to capture persistent characteristics of the entities in the sample.
This means that the characteristic of the entity that is captured by the given variable is assumed to remain reasonably stable over time.

In this chapter, we discuss a technique that we call persistence analysis. We use persistence analysis to examine whether a given characteristic of the entities in our sample is in fact persistent. Persistence analysis can also be used to examine the ability of the variable in question to capture the desired characteristic of the entity

First Step: calculating cross-sectional correlations between the given variable X measured a certain number of periods apart.

Second Step: calculating the time-series average of each of these cross-sectional correlations.

In [314]:
def calculate_cross_sectional_persistence(df, time_column, value_columns, entity_column=None, max_tau=5):
    """
    Calculate the cross-sectional Pearson correlations for multiple variables measured tau periods apart for multiple tau values.
    
    Args:
        df (pd.DataFrame): The data frame containing the data.
        time_column (str): The name of the column representing time periods.
        value_columns (list of str): The names of the columns representing the values of the variables.
        entity_column (str or None): The name of the column representing the entities. If None, calculate persistence without entity grouping.
        max_tau (int): The maximum number of periods apart to measure persistence.
    
    Returns:
        dict of pd.DataFrame: A dictionary containing the persistence correlations for each variable.
    """
    persistence_results = {}

    for value_column in value_columns:
        # Ensure the dataframe is sorted by the time column (and entity column if provided)
        if entity_column:
            df = df.sort_values(by=[time_column, entity_column]).reset_index(drop=True)
        else:
            df = df.sort_values(by=[time_column]).reset_index(drop=True)
        
        # Create a dictionary to store the results
        results = {f't+{tau}': [] for tau in range(1, max_tau + 1)}
        results['Year'] = []

        # Get unique time periods
        unique_times = df[time_column].unique()

        # Loop over each time period
        for t in unique_times:
            period_df = df[df[time_column] == t]
            
            if len(period_df) == 0:
                continue

            correlations = []
            
            for tau in range(1, max_tau + 1):
                future_time = t + tau
                
                if future_time not in unique_times:
                    correlations.append(np.nan)
                    continue
                
                shifted_df = df[df[time_column] == future_time]
                
                if entity_column:
                    # Merge on entities to ensure we only consider those with valid values for both t and t+tau
                    merged_df = pd.merge(period_df, shifted_df, on=entity_column, suffixes=('', f'_shifted_{tau}'))
                else:
                    # If no entity column, just ensure both periods have data
                    merged_df = pd.concat([period_df.reset_index(), shifted_df.reset_index()], axis=1, keys=['t', 't+tau'])

                if len(merged_df) == 0:
                    correlations.append(np.nan)
                    continue
                
                # Calculate means
                X_t = merged_df[value_column] if entity_column else merged_df[('t', value_column)]
                X_t_tau = merged_df[f'{value_column}_shifted_{tau}'] if entity_column else merged_df[('t+tau', value_column)]
                mean_X_t = X_t.mean()
                mean_X_t_tau = X_t_tau.mean()

                # Calculate numerator and denominator separately
                numerator = ((X_t - mean_X_t) * (X_t_tau - mean_X_t_tau)).sum()
                denominator = np.sqrt(((X_t - mean_X_t) ** 2).sum() * ((X_t_tau - mean_X_t_tau) ** 2).sum())
                
                if denominator == 0:
                    correlations.append(np.nan)
                else:
                    pearson_corr = numerator / denominator
                    correlations.append(pearson_corr)
            
            results['Year'].append(t)
            for tau, corr in zip(range(1, max_tau + 1), correlations):
                results[f't+{tau}'].append(corr)
        
        results_df = pd.DataFrame(results)
        results_df.set_index('Year', inplace=True)
        persistence_results[value_column] = results_df

    return persistence_results

def calculate_average_persistence(persistence_results, max_tau):
    """
    Calculate the time-series average of the periodic cross-sectional correlations for multiple variables.
    
    Args:
        persistence_results (dict of pd.DataFrame): A dictionary containing the periodic cross-sectional correlations for each variable.
        max_tau (int): The maximum number of periods apart to measure persistence.
    
    Returns:
        pd.DataFrame: A data frame containing the average persistence values for each variable and each lag.
    """
    avg_persistence = {f't+{tau}': [] for tau in range(1, max_tau + 1)}
    avg_persistence['Variable'] = []

    for variable, persistence_df in persistence_results.items():
        avg_persistence['Variable'].append(variable)
        for tau in range(1, max_tau + 1):
            avg_persistence[f't+{tau}'].append(persistence_df[f't+{tau}'].mean())

    avg_persistence_df = pd.DataFrame(avg_persistence)
    avg_persistence_df.set_index('Variable', inplace=True)
    return avg_persistence_df




Average Persistence with Entity Column:
               t+1       t+2       t+3       t+4       t+5
Variable                                                  
beta     -0.012372  0.095060  0.222180  0.313634 -0.391324
Size     -0.077550 -0.379835  0.267808  0.534534 -0.420979
BM       -0.120001 -0.198523  0.396845 -0.014092 -0.130984

Average Persistence without Entity Column:
               t+1       t+2       t+3       t+4       t+5
Variable                                                  
beta     -0.084641 -0.096246  0.034608  0.035583 -0.006334
Size     -0.124825 -0.473607  0.182061  0.390492 -0.433567
BM       -0.118471 -0.340782  0.142714  0.067371  0.007421


In [ ]:
# Example usage
# Generate example data
years = list(range(1994, 2011))
entities = ['A', 'B', 'C']
data = []

for year in years:
    for entity in entities:
        data.append({'Year': year, 'entity': entity, 'beta': np.random.randint(10, 100),
                     'Size': np.random.randint(50, 150), 'BM': np.random.randint(20, 120)})

# Make data sparse by removing some entries for the last few years
df = pd.DataFrame(data)
df = df[~((df['Year'] >= 2008) & (df['entity'] == 'C'))]

variables = ['beta', 'Size', 'BM']
max_tau = 5

# Calculate persistence correlations with entity column
persistence_results_with_entity = calculate_cross_sectional_persistence(df, 'Year', variables, 'entity', max_tau)

# Calculate average persistence with entity column
average_persistence_with_entity = calculate_average_persistence(persistence_results_with_entity, max_tau)
print("Average Persistence with Entity Column:")
print(average_persistence_with_entity)

# Calculate persistence correlations without entity column
persistence_results_without_entity = calculate_cross_sectional_persistence(df.drop(['entity'], axis=1), 'Year', variables, entity_column=None, max_tau=max_tau)

# Calculate average persistence without entity column
average_persistence_without_entity = calculate_average_persistence(persistence_results_without_entity, max_tau)
print("\nAverage Persistence without Entity Column:")
print(average_persistence_without_entity)

As might be expected, the correlations between characteristics measured at longer lags 𝜏 tend to be lower than the correlations measured at shorter lags, although this is not always the case.
However, it is worth noting that for years $t$ toward the end of the sample, in some cases the persistence values are missing.

Although periodic cross-sectional persistence values are quite informative, they are quite difficult to read and draw conclusions from. We therefore want to summarize these periodic values more succinctly
As with the other analyses we discuss, the main objective is to understand the persistence of the variable X in the average cross section. We therefore summarize the results by simply taking the time-series average of the periodic cross-sectional correlations. We denote these average persistence values

In general, a higher degree of time-lagged cross-sectional correlation in the given variable is indicative of higher persistence, although there are several caveats to this that must be understood to properly make use of this technique.

Exactly what qualifies as low persistence is not perfectly well defined and depends on how long the lag is between the times of measurement (𝜏), how persistent the actual 
There are generally two reasons that a variable may exhibit low or zero persistence. 
The first is that the characteristic being measured is in fact not persistent.
The second is that the variable used to proxy for the given characteristic does a poor job at measuring the characteristic under examination.
In this case, even if the given characteristic of the entities in the sample is highly cross-sectionally persistent, the failure of the variable X to capture cross-sectional variation in this characteristic will cause the persistence analysis to generate a low value of 𝜌𝜏 (X).

In the end, however, regardless of the reason for the lack of persistence in X, if X is intended to capture a persistent characteristic of a firm, but X does not exhibit persistence, then X is not a good measure of the characteristic of interest.

When the persistence analysis produces high values of 𝜌𝜏 (X), this very likely indicates both that the characteristic in question is in fact persistent and that the variable X does a good job at measuring the characteristic.

There are two caveats with this statement that must be addressed. 

The first is that it is possible that the variable X is unintentionally capturing some persistent characteristic of the entities in the sample that is different from the characteristic that X is designed to capture. Thus, perhaps a more correct statement is that high values of 𝜌𝜏 (X) indicate that whatever characteristic is being captured by X is in fact persistent.
If X does in fact capture the intended characteristic, then we can conclude that the characteristic is in fact persistent. 
Therefore, assuming sufficient effort has been devoted to designing the calculation of X such that it can reasonably be expected to capture the intended characteristic, high values of 𝜌𝜏 (X) are interpreted as indicating that the given characteristic is in fact persistent.

The second, and much more important, caveat associated with concluding that a characteristic is persistent is that in many cases there is a mechanical reason related to the calculation of X that would result in strong cross-sectional correlation between Xt and Xt+𝜏 even if the characteristic in question is not persistent. In most cases, the reason for such a mechanical effect is that some subset of the data used to calculate X at times t and t + 𝜏 are the same. 

While certainly none of the variables studied in asset pricing research are perfectly persistent, different variables exhibit different degrees of persistence.

If the persistence of X2 at lag 𝜏 = 𝜏2 is less than that of X1 at lag 𝜏 = 𝜏1, the results are a bit more challenging to interpret, but it can generally be taken to mean that the decay in the persistence over a period of 𝜏2 − 𝜏1 periods is substantial enough to overcome any additional benefit of using 𝜏2 periods of data, compared to 𝜏1, to calculate X. If this is the case, it may also be an indication that using a full 𝜏2 periods of data is too long a measurement period because the given characteristic of the firm does in fact change substantially over periods of length 𝜏2.

## Ch5. Portfolio Analysis

The purpose of portfolio analysis is to detect the variables for predicting cross-sectional variation of stock returns

Understanding variation in characteristics of the stock across the different portfolio.

Benefit of the portfolio analysis: nonparametric technique
Drawback of the portfolio analysis: difficult to control for a large number of variables


### Univariate Portfolio Analysis

only one sort variable X.
The objective of the analysis is to assess the cross-sectional relation between X and outcome variable Y.
The univariate portfolio analysis has four steps:
1. Caculate the breakpoints that will be used to divide the sample into portfolio
2. Using these breakpoints to form the portfolio
3. Caculate the average value of the outcome variable Y
4. Examine variation in these average vlaues of Y across the different portfolio

#### Breakpoints

In [329]:
def cal_bp(df, time_column, value_column, num_portfolios, custom_percentiles=None):
    """
    Calculate breakpoints for a given variable based on specified quantiles for univariate portfolio analysis.
    
    Args:
        df (pd.DataFrame): The data frame containing the data.
        time_column (str): The name of the column representing time periods.
        value_column (str): The name of the column representing the values to calculate breakpoints for.
        num_portfolios (int): The number of portfolios to be formed each time period.
        custom_percentiles (list of float, optional): Custom percentiles to calculate breakpoints. If None, evenly spaced percentiles are used.
    
    Returns:
        pd.DataFrame: A data frame containing the breakpoints for each time period.
    """
    if custom_percentiles:
        # Ensure custom_percentiles are in the range (0, 100)
        assert all(0 < p < 100 for p in custom_percentiles), "Percentiles must be between 0 and 100"
        # Convert custom_percentiles to a fraction of 1
        quantiles = [p / 100 for p in custom_percentiles]
    else:
        # Calculate evenly spaced quantiles
        quantiles = [k / num_portfolios for k in range(1, num_portfolios)]
    
    breakpoints = df.groupby(time_column)[value_column].quantile(quantiles).unstack()
    breakpoints.columns = [f'B{k+1}' for k in range(len(quantiles))]
    return breakpoints

#### Portfolio Formation

In [325]:
def assign_portfolios(df, time_column, value_column, breakpoints):
    """
    Assign each data point to a portfolio based on the calculated breakpoints.
    
    Args:
        df (pd.DataFrame): The data frame containing the data.
        time_column (str): The name of the column representing time periods.
        value_column (str): The name of the column representing the values to assign to portfolios.
        breakpoints (pd.DataFrame): The breakpoints for each time period.
    
    Returns:
        pd.DataFrame: The original data frame with an additional column for the assigned portfolios.
    """
    def get_portfolio(row, breakpoints):
        period_breakpoints = breakpoints.loc[row[time_column]]
        for i in range(len(period_breakpoints)):
            if row[value_column] <= period_breakpoints.iloc[i]:
                return i + 1
        return len(period_breakpoints) + 1

    df['Portfolio'] = df.apply(lambda row: get_portfolio(row, breakpoints), axis=1)
    return df

#### Average Portfolio Values

Value-weighting is most appropriate when the entities in the analysis are stocks. In such cases, the results of equal-weighted analyses are indicative of phenomena for the average stock. The results of value-weighted analyses account for the importance, from the point of view of the stock market as a whole, of each individual stock relative to the other stocks in the given portfolio. When the outcome variables (Y) is the future stock return, the results of value-weighted analyses are generally considered to be more indicative of return that an investor would have realized by implementing the portfolio in question. The reason for this is that value-weighted portfolios have large weights on stocks with large market capitalizations, which tend to be highly liquid. The returns of equal-weighted portfolios are potentially driven by the low-market capitalization stocks in the portfolio, which are more expensive to trade. The result is often that the average return indicated by the portfolio analysis cannot be realized by an actual investor because of transaction costs.

In [357]:
def cal_portfolio(df, time_column, portfolio_column, outcome_column, weight_column=None):
    """
    Calculate the average value of the outcome variable for each portfolio.
    
    Args:
        df (pd.DataFrame): The data frame containing the data.
        time_column (str): The name of the column representing time periods.
        portfolio_column (str): The name of the column representing the assigned portfolios.
        outcome_column (str): The name of the column representing the outcome variable.
        weight_column (str, optional): The name of the column representing the weights. If None, equal-weighted averages are calculated.
    
    Returns:
        pd.DataFrame: A data frame containing the average values of the outcome variable for each portfolio.
    """
    if weight_column:
        # Calculate weighted sum and sum of weights for each portfolio
        weighted_sum = df.groupby([time_column, portfolio_column]).apply(
            lambda x: (x[outcome_column] * x[weight_column]).sum()
        )
        sum_weights = df.groupby([time_column, portfolio_column])[weight_column].sum()
        portfolio_averages = (weighted_sum / sum_weights).unstack()
    else:
        portfolio_averages = df.groupby([time_column, portfolio_column])[outcome_column].mean().unstack()
    
    # Calculate the difference between the highest and lowest portfolios
    portfolio_averages['Diff'] = portfolio_averages.iloc[:, -1] - portfolio_averages.iloc[:, 0]
    
    return portfolio_averages

In [362]:
def cal_portfolio(df, time_column, portfolio_column, outcome_column, weight_column=None):
    """
    Calculate the average value of the outcome variable for each portfolio.
    
    Args:
        df (pd.DataFrame): The data frame containing the data.
        time_column (str): The name of the column representing time periods.
        portfolio_column (str): The name of the column representing the assigned portfolios.
        outcome_column (str): The name of the column representing the outcome variable.
        weight_column (str, optional): The name of the column representing the weights. If None, equal-weighted averages are calculated.
    
    Returns:
        pd.DataFrame: A data frame containing the average values of the outcome variable for each portfolio.
    """
    
    if weight_column:
        df['weighted_outcome'] = df[outcome_column] * (df[weight_column] / df.groupby([time_column, portfolio_column])[weight_column].transform('sum'))
        portfolio_averages = df.groupby([time_column, portfolio_column])['weighted_outcome'].sum().unstack()
    else:
        portfolio_averages = df.groupby([time_column, portfolio_column])[outcome_column].mean().unstack()
    
    # Calculate the difference between the highest and lowest portfolios
    portfolio_averages['Diff'] = portfolio_averages.max(axis=1) - portfolio_averages.min(axis=1)
    
    return portfolio_averages

#### Summarize the Results

In [351]:
def calculate_time_series_means(portfolio_averages):
    """
    Calculate the time-series means of the period average values of the outcome variable for each portfolio and the difference portfolio.
    
    Args:
        portfolio_averages (pd.DataFrame): A data frame containing the average values of the outcome variable for each portfolio.
    
    Returns:
        pd.Series: A series containing the time-series mean values for each portfolio and the difference portfolio.
    """
    # Calculate the time-series mean for each portfolio
    time_series_means = portfolio_averages.mean()
    
    # Separate the difference portfolio mean
    diff_mean = time_series_means.pop('Diff')
    
    return time_series_means, diff_mean

In [368]:
def calculate_portfolio_averages(df, time_column, portfolio_column, outcome_column, weight_column=None):
    """
    Calculate the average value of the outcome variable for each portfolio.
    
    Args:
        df (pd.DataFrame): The data frame containing the data.
        time_column (str): The name of the column representing time periods.
        portfolio_column (str): The name of the column representing the assigned portfolios.
        outcome_column (str): The name of the column representing the outcome variable.
        weight_column (str, optional): The name of the column representing the weights. If None, equal-weighted averages are calculated.
    
    Returns:
        pd.DataFrame: A data frame containing the average values of the outcome variable for each portfolio.
    """
    def portfolio_return(group, outcome_column, weight_column=None):
        """
        Calculate portfolio return for a group of data.
        
        Args:
            group (pd.DataFrame): The data frame containing the data for a group.
            outcome_column (str): The name of the column representing the outcome variable.
            weight_column (str, optional): The name of the column representing the weights. If None, equal-weighted returns are calculated.
        
        Returns:
            float: The portfolio return.
        """
        ret = group[outcome_column]
        if weight_column:
            wgt = group[weight_column]
            wgt = wgt / wgt.sum()  # normalize weights to sum to 1
            r_p = np.dot(ret, wgt)
        else:
            r_p = ret.mean()
        return r_p
    
    if weight_column:
        portfolio_averages = df.groupby([time_column, portfolio_column]).apply(
            portfolio_return, outcome_column=outcome_column, weight_column=weight_column
        ).unstack()
    else:
        portfolio_averages = df.groupby([time_column, portfolio_column])[outcome_column].mean().unstack()
    
    # Calculate the difference between the highest and lowest portfolios
    portfolio_averages['Diff'] = portfolio_averages.iloc[:, -1] - portfolio_averages.iloc[:, 0]
    
    return portfolio_averages

In [374]:
# Example usage with detailed example data
np.random.seed(0)
years = list(range(1994, 2011))
entities = ['A', 'B', 'C', 'D']
data = []

# Creating example data with different market capitalizations and returns
for year in years:
    for entity in entities:
        data.append({
            'Year': year,
            'entity': entity,
            'X': np.random.rand(),  # Sort variable (e.g., beta)
            'Y': np.random.rand(),  # Outcome variable (e.g., return)
            'MktCap': np.random.rand() * 1000  # Market capitalization
        })

df = pd.DataFrame(data)

# Specify the number of portfolios
num_portfolios = 5

In [375]:
# Calculate breakpoints
breakpoints = cal_bp(df, 'Year', 'X', num_portfolios)
print("Breakpoints:")
print(breakpoints)

Breakpoints:
            B1        B2        B3        B4
Year                                        
1994  0.415929  0.459046  0.523424  0.546455
1995  0.375678  0.610067  0.736134  0.786557
1996  0.206043  0.325331  0.507658  0.718928
1997  0.647221  0.672874  0.679584  0.688145
1998  0.235526  0.265719  0.303001  0.364698
1999  0.121349  0.142340  0.154812  0.423779
2000  0.193359  0.319765  0.390638  0.639262
2001  0.228718  0.368175  0.516995  0.626492
2002  0.347573  0.578228  0.584442  0.645985
2003  0.437996  0.627778  0.767293  0.826842
2004  0.181724  0.360860  0.573208  0.676496
2005  0.274727  0.412224  0.546211  0.615365
2006  0.429115  0.653686  0.768067  0.851509
2007  0.460900  0.739108  0.790266  0.826713
2008  0.459730  0.742732  0.780956  0.847451
2009  0.208613  0.252703  0.297135  0.401420
2010  0.134927  0.247046  0.429606  0.572813


In [376]:
# Assign portfolios
df_with_portfolios = assign_portfolios(df, 'Year', 'X', breakpoints)
print("\nData with Portfolios:")
print(df_with_portfolios.head())



Data with Portfolios:
   Year entity         X         Y      MktCap  Portfolio
0  1994      A  0.548814  0.715189  602.763376          5
1  1994      B  0.544883  0.423655  645.894113          4
2  1994      C  0.437587  0.891773  963.662761          2
3  1994      D  0.383442  0.791725  528.894920          1
4  1995      A  0.568045  0.925597   71.036058          2


In [377]:

# Calculate equal-weighted portfolio averages
equal_weighted_averages = calculate_portfolio_averages(df_with_portfolios, 'Year', 'Portfolio', 'Y')
print("\nEqual-Weighted Portfolio Averages:")
print(equal_weighted_averages)


Equal-Weighted Portfolio Averages:
Portfolio         1         2         4         5      Diff
Year                                                       
1994       0.791725  0.891773  0.423655  0.715189 -0.076536
1995       0.020218  0.925597  0.870012  0.461479  0.441261
1996       0.639921  0.774234  0.018790  0.521848 -0.118073
1997       0.616934  0.210383  0.359508  0.060225 -0.556709
1998       0.161310  0.466311  0.363711  0.988374  0.827064
1999       0.976459  0.196582  0.110375  0.097101 -0.879358
2000       0.282807  0.118728  0.064147  0.604846  0.322039
2001       0.575946  0.667410  0.265389  0.289406 -0.286540
2002       0.677817  0.592042  0.020108  0.962189  0.284372
2003       0.952749  0.881735  0.396506  0.699479 -0.253270
2004       0.301575  0.618015  0.423855  0.501324  0.199750
2005       0.298282  0.435865  0.574325  0.431418  0.133136
2006       0.868126  0.123820  0.703889  0.714241 -0.153885
2007       0.697429  0.866382  0.569101  0.011714 -0.685715
2008

In [378]:
# Calculate value-weighted portfolio averages
value_weighted_averages = calculate_portfolio_averages(df_with_portfolios, 'Year', 'Portfolio', 'Y', 'MktCap')
print("\nValue-Weighted Portfolio Averages:")
print(value_weighted_averages)



Value-Weighted Portfolio Averages:
Portfolio         1         2         4         5      Diff
Year                                                       
1994       0.791725  0.891773  0.423655  0.715189 -0.076536
1995       0.020218  0.925597  0.870012  0.461479  0.441261
1996       0.639921  0.774234  0.018790  0.521848 -0.118073
1997       0.616934  0.210383  0.359508  0.060225 -0.556709
1998       0.161310  0.466311  0.363711  0.988374  0.827064
1999       0.976459  0.196582  0.110375  0.097101 -0.879358
2000       0.282807  0.118728  0.064147  0.604846  0.322039
2001       0.575946  0.667410  0.265389  0.289406 -0.286540
2002       0.677817  0.592042  0.020108  0.962189  0.284372
2003       0.952749  0.881735  0.396506  0.699479 -0.253270
2004       0.301575  0.618015  0.423855  0.501324  0.199750
2005       0.298282  0.435865  0.574325  0.431418  0.133136
2006       0.868126  0.123820  0.703889  0.714241 -0.153885
2007       0.697429  0.866382  0.569101  0.011714 -0.685715
2008

C:\Users\YeonChan Kang\AppData\Local\Temp\ipykernel_72644\3222318002.py:37: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  portfolio_averages = df.groupby([time_column, portfolio_column]).apply(


In [379]:
# Calculate time-series means for equal-weighted averages
time_series_means_eq, diff_mean_eq = calculate_time_series_means(equal_weighted_averages)
print("\nTime-Series Means (Equal-Weighted):")
print(time_series_means_eq)
print("\nDifference Mean (Equal-Weighted):")
print(diff_mean_eq)


Time-Series Means (Equal-Weighted):
Portfolio
1    0.536305
2    0.577594
4    0.353702
5    0.513583
dtype: float64

Difference Mean (Equal-Weighted):
-0.022722263559856775


In [380]:
# Calculate time-series means for value-weighted averages
time_series_means_val, diff_mean_val = calculate_time_series_means(value_weighted_averages)
print("\nTime-Series Means (Value-Weighted):")
print(time_series_means_val)
print("\nDifference Mean (Value-Weighted):")
print(diff_mean_val)


Time-Series Means (Value-Weighted):
Portfolio
1    0.536305
2    0.577594
4    0.353702
5    0.513583
dtype: float64

Difference Mean (Value-Weighted):
-0.022722263559856775


In [1]:


# Calculate time-series means for equal-weighted averages
def calculate_time_series_means(portfolio_averages):
    """
    Calculate the time-series means of the period average values of the outcome variable for each portfolio and the difference portfolio.
    
    Args:
        portfolio_averages (pd.DataFrame): A data frame containing the average values of the outcome variable for each portfolio.
    
    Returns:
        pd.Series: A series containing the time-series mean values for each portfolio and the difference portfolio.
    """
    time_series_means = portfolio_averages.mean()
    diff_mean = time_series_means.pop('Diff')
    return time_series_means, diff_mean

time_series_means_eq, diff_mean_eq = calculate_time_series_means(equal_weighted_averages)
print("\nTime-Series Means (Equal-Weighted):")
print(time_series_means_eq)
print("\nDifference Mean (Equal-Weighted):")
print(diff_mean_eq)

time_series_means_val, diff_mean_val = calculate_time_series_means(value_weighted_averages)
print("\nTime-Series Means (Value-Weighted):")
print(time_series_means_val)
print("\nDifference Mean (Value-Weighted):")
print(diff_mean_val)

NameError: name 'equal_weighted_averages' is not defined

## Ch6. Fama-Macbeth regression analysis